## 2 Whisper Baseline

Before fine-tuning our own model, we will first decode with the whisper model to get a baseline word error rate.

In [1]:
import os
import torch
import whisper
import pandas as pd
from tqdm import tqdm
from jiwer import wer, cer
import warnings
import setproctitle
from helpers import get_best_gpu
warnings.filterwarnings("ignore")

PROCESS_NAME = "romansh-asr"
setproctitle.setproctitle(PROCESS_NAME)

DATA_PATH = "romansh-data/sursilvan-mini/"
WHISPER_MODEL = "medium"
BATCH_SIZE = 8
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
OUTPUT_FILE = "whisper_baseline_results.txt"

Checking GPU availability

In [2]:
print("="*60)
print("Whisper Baseline for Romansh ASR")
print("="*60)
print(f"Model: {WHISPER_MODEL}")
print(f"Device: {DEVICE}")
print(f"Data path: {DATA_PATH}")
print("="*60)

if DEVICE == "cuda":
    best_gpu = get_best_gpu()
    torch.cuda.set_device(best_gpu)
    print(f"GPU: {torch.cuda.get_device_name(best_gpu)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(best_gpu).total_memory / 1e9:.2f} GB")
else:
    print("CUDA not available - running on CPU (this will be slow!)")

Whisper Baseline for Romansh ASR
Model: medium
Device: cuda
Data path: romansh-data/sursilvan-mini/
📊 Selected GPU 7 with 24121 MiB free memory


GPU: NVIDIA GeForce RTX 3090
GPU Memory: 25.30 GB


Defining some helpful helper functions

In [9]:
def calculate_metrics(references, hypotheses):
    try:
        wer_score = wer(references, hypotheses)
        cer_score = cer(references, hypotheses)
        return wer_score, cer_score
    except Exception as e:
        print(f"Warning: Error calculating metrics: {e}")
        return None, None

Loading the whisper model

In [10]:
model = whisper.load_model(WHISPER_MODEL, device=DEVICE)

Decoding with the whisper model

In [11]:
def transcribe_all_with_language(model, audio_paths):
    transcriptions = []
    detected_languages = []
    
    for audio_path in tqdm(audio_paths, desc="Transcribing audio files"):
        try:
            if not os.path.exists(audio_path):
                transcriptions.append("")
                detected_languages.append("missing")
                continue
            
            result = model.transcribe(
                audio_path,
                task="transcribe",
                fp16=torch.cuda.is_available(),
                verbose=False
            )
            
            transcriptions.append(result['text'].strip())
            detected_languages.append(result.get('language', 'unknown'))
            
        except Exception as e:
            transcriptions.append("")
            detected_languages.append("error")
    
    return transcriptions, detected_languages

Processing all splits

In [12]:
splits = ['train', 'validated', 'test']
all_results = {}
language_summary = {}

for split in splits:
    print(f"\n{'='*50}")
    print(f"Processing {split} split")
    print(f"{'='*50}")
    
    tsv_path = os.path.join(DATA_PATH, f"{split}.tsv")
    clips_path = os.path.join(DATA_PATH, "clips")
    
    if not os.path.exists(tsv_path):
        print(f"{split}.tsv not found, skipping...")
        continue
    
    df = pd.read_csv(tsv_path, sep='\t')
    
    audio_paths = [os.path.join(clips_path, path) for path in df['path']]
    
    existing_indices = [i for i, path in enumerate(audio_paths) if os.path.exists(path)]
    missing_count = len(audio_paths) - len(existing_indices)
    
    if missing_count > 0:
        print(f"{missing_count} audio files missing, filtering them out")
        df = df.iloc[existing_indices].reset_index(drop=True)
        audio_paths = [audio_paths[i] for i in existing_indices]
    
    print(f"Total utterances: {len(df)}")
    
    print(f"\nRunning Whisper transcription...")
    hypotheses, detected_langs = transcribe_all_with_language(model, audio_paths)
    
    df['detected_language'] = detected_langs
    
    references = df['sentence'].tolist()
    wer_score, cer_score = calculate_metrics(references, hypotheses)
    
    all_results[split] = {
        'wer': wer_score,
        'cer': cer_score,
        'count': len(df),
        'references': references[:5],
        'hypotheses': hypotheses[:5],
        'languages': detected_langs[:5]
    }
    
    lang_counts = df['detected_language'].value_counts()
    language_summary[split] = lang_counts
    
    print(f"\nResults for {split}:")
    print(f"   Utterances: {len(df)}")
    if wer_score is not None:
        print(f"   WER: {wer_score:.4f} ({wer_score*100:.2f}%)")
        print(f"   CER: {cer_score:.4f} ({cer_score*100:.2f}%)")
    else:
        print("   No valid reference-hypothesis pairs")
    
    print(f"\nLanguage detection distribution:")
    for lang, count in lang_counts.items():
        print(f"   {lang}: {count} ({count/len(df)*100:.1f}%)")


Processing train split
Total utterances: 92

Running Whisper transcription...


Transcribing audio files:   0%|          | 0/92 [00:00<?, ?it/s]

Detected language: Slovenian


Transcribing audio files:   1%|          | 1/92 [00:02<03:28,  2.29s/it]

Detected language: German


...

Detected language: Yiddish


Transcribing audio files:  99%|█████████▉| 93/94 [10:19<00:05,  5.41s/it]

Detected language: Yiddish


Transcribing audio files: 100%|██████████| 94/94 [10:24<00:00,  6.65s/it]


Results for test:
   Utterances: 94
   WER: 0.9815 (98.15%)
   CER: 0.6046 (60.46%)

Language detection distribution:
   de: 33 (35.1%)
   sl: 22 (23.4%)
   oc: 9 (9.6%)
   it: 6 (6.4%)
   la: 5 (5.3%)
   sq: 3 (3.2%)
   nl: 3 (3.2%)
   cy: 3 (3.2%)
   yi: 3 (3.2%)
   br: 1 (1.1%)
   hu: 1 (1.1%)
   en: 1 (1.1%)
   pt: 1 (1.1%)
   he: 1 (1.1%)
   lt: 1 (1.1%)
   fr: 1 (1.1%)


Save results to file

In [13]:
print(f"\nSaving results to {OUTPUT_FILE}")
with open(OUTPUT_FILE, 'w') as f:
    f.write("="*60 + "\n")
    f.write("Whisper Baseline Results\n")
    f.write(f"Model: {WHISPER_MODEL}\n")
    f.write(f"Date: {pd.Timestamp.now()}\n")
    f.write("="*60 + "\n\n")
    
    for split, results in all_results.items():
        f.write(f"\n{split.upper()} Split:\n")
        f.write(f"  Utterances: {results['count']}\n")
        if results['wer'] is not None:
            f.write(f"  WER: {results['wer']:.4f}\n")
            f.write(f"  CER: {results['cer']:.4f}\n")
        
        f.write("\n  Language Distribution:\n")
        for lang, count in language_summary[split].items():
            f.write(f"    {lang}: {count}\n")
        
        f.write("\n  Example transcriptions:\n")
        for i, (ref, hyp, lang) in enumerate(zip(results['references'], 
                                                results['hypotheses'], 
                                                results['languages'])):
            f.write(f"    Ref {i+1}: {ref}\n")
            f.write(f"    Hyp {i+1}: {hyp}\n")
            f.write(f"    Lang {i+1}: {lang}\n\n")


Saving results to whisper_baseline_results.txt


Show summary

In [ ]:
print("\n" + "="*60)
print("Summary")
print("="*60)

total_utterances = 0
for split in splits:
    if split in all_results:
        results = all_results[split]
        print(f"\n{split.upper()}:")
        print(f"  Utterances: {results['count']}")
        total_utterances += results['count']
        if results['wer'] is not None:
            print(f"  WER: {results['wer']:.4f}")
            print(f"  CER: {results['cer']:.4f}")

print(f"\nTotal utterances processed: {total_utterances}")
print("="*60)
print("Done!")


Summary

TRAIN:
  Utterances: 92
  WER: 0.9700
  CER: 0.5693

VALIDATED:
  Utterances: 93
  WER: 1.0050
  CER: 0.6449

TEST:
  Utterances: 94
  WER: 0.9815
  CER: 0.6046

Total utterances processed: 279
Done!


: 